In [2]:
!pip install ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.10.2-py3-none-any.whl size=29088 sha256=0be1368f8cc2b99147ba4f518014dc358e59ecdce5ce87420311ca9e73e96574
  Stored in directory: /root/.cache/pip/wheels/47/51/06/380dc516ea78621870b93ff65527c251afdfdc5fa9d7f4d248
Successfully built ta


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#call library

import os
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Dense, LSTM, Dropout, Input, RepeatVector
from tensorflow.keras.layers import Bidirectional, TimeDistributed
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import ta

#from preprocesslib import preprocessEach, makeY

In [ ]:
import os
import pandas as pd
import numpy as np

'''
이런식으로 사용하면 됩니다.
from preprocesslib import preprocessEach, makeY
X = preprocessEach('overlap')
X bootstrap해서 -> X_final만든 후에
y = makeY('overlap', len(X_final))
'''

def preprocessEach(folder_name):
   # set file path
    # folder_path = f'/content/drive/MyDrive/research/data/{folder_name}_clean'
    folder_path = f'../data/{folder_name}_clean'


    # load file name in folder
    file_names = os.listdir(folder_path)
    data_array = []

    for file in file_names:
        # set file path
        file_path = os.path.join(folder_path, file)
        
        # load csv file as dataframe
        df = pd.read_csv(file_path, index_col=0)
        df.index = pd.to_datetime(df.index)

        #preprocessing column
        df['open-close'] = df['Open'] - df['Close'] #추가
        df['open-close'] = df['open-close'] + abs(min(df['open-close'])) #추가
        df['MA5'] = df['Close'].rolling(5).mean() #추가
        df['Diff'] = df['High'] - df['Low']
        df['RSI'] = ta.momentum.rsi(df['Close'])
        df['MACD'] = ta.trend.macd(df['Close'])


        df = df[['Close','Volume','Diff','open-close','MA5', 'Search','MACD','RSI']]
        df.fillna(0,inplace=True)



        #log transform
        df = np.log1p(df)
        # df.reset_index(inplace=True)
        df.fillna(0,inplace=True)

        #insert list to array
        data_array.append(df)

    return np.array(data_array)

def makeY(folder_name, filelength):

    if folder_name=='growth':
        yGrowths = np.ones(filelength)
        yValues = np.zeros(filelength)
    elif folder_name=='value':
        yGrowths = np.zeros(filelength)
        yValues = np.ones(filelength)
    elif folder_name=='overlap':
        yGrowths = np.ones(filelength)
        yValues = np.ones(filelength)

    y = np.column_stack([yGrowths.T, yValues.T])
    return y

In [ ]:
xGrowths = preprocessEach('growth')
xValues = preprocessEach('value')
xOverlaps = preprocessEach('overlap')

In [ ]:
#bootstrap
n_samples = len(xValues)
diff = len(xValues) - len(xGrowths)
xGrowths_bootstrapped = xGrowths[np.random.choice(len(xGrowths), size=diff, replace=True)]
xGrowths = np.concatenate((xGrowths, xGrowths_bootstrapped), axis=0)

diff = len(xValues) - len(xOverlaps)
xOverlaps_bootstrapped = xOverlaps[np.random.choice(len(xOverlaps), size=diff, replace=True)]
xOverlaps = np.concatenate((xOverlaps, xOverlaps_bootstrapped), axis=0)

In [ ]:
#make y values
yGrowths = makeY('growth', n_samples)
yValues = makeY('value', n_samples)
yOverlaps = makeY('overlap', n_samples)

In [ ]:
x = np.concatenate((xGrowths, xValues, xOverlaps), axis=0)
y = np.concatenate((yGrowths, yValues, yOverlaps), axis=0)

In [ ]:
#minmax scaling by element
scaler = MinMaxScaler()
for i in range(8):
    x[:,:,i] = scaler.fit_transform(x[:,:,i])

In [ ]:
X_8 = pd.DataFrame(x)

In [ ]:
def splitData(x, y, train_ratio=0.8):
    # shuffle data
    np.random.seed(2023)
    permutation = np.random.permutation(len(x))
    x = x[permutation]
    y = y[permutation]

    # split data
    n_train_samples = int(train_ratio * len(x))
    x_train, x_test = x[:n_train_samples], x[n_train_samples:]
    y_train, y_test = y[:n_train_samples], y[n_train_samples:]

    return x_train, x_test, y_train, y_test

In [ ]:
X_train, X_test, Y_train, Y_test = splitData(x,y)
# X_train.isnull().any()
# X_train = X_train +1

In [ ]:
# 1) 파라미터 설정
# n_step = int(input('시계열 recurrent timestep 설정: '))
n_hidden = 3
# MASHORT = int(input('종가 단기이평 기간 설정: '))
# MALONG = int(input('종가 장기이평 기간 설정: '))

In [ ]:
from tensorflow.keras.optimizers.legacy import Adam
# 4) LSTM 오토인코더 모델 네트워크 설정
X_input = Input(batch_shape=(None, X_train.shape[1], X_train.shape[2])) # Input
X_encoder = Bidirectional(LSTM(n_hidden, return_sequences=True), merge_mode='sum')(X_input) # 인코더
y_decoder = Bidirectional(LSTM(n_hidden, return_sequences=True), merge_mode='sum')(X_encoder) # 디코더
y_output = TimeDistributed(Dense(X_train.shape[2]))(y_decoder) # 출력

# 5) 모델 구성 및 학습
model = Model(X_input, y_output)
print("=========== 모델 전체 구조 ===========")
print(model.summary())
print()
model_encoder = Model(X_input, X_encoder)
print("=========== 인코더 모델 구조 ===========")
print(model_encoder.summary())
print()

# 모델 학습
EPOCHS = int(100)
BATCH = int(128)
model.compile(loss='mse', optimizer=Adam(learning_rate=0.0001))
hist = model.fit(X_train, X_train,
                 epochs=EPOCHS,
                 batch_size=BATCH)

In [ ]:
n_hidden = 3
X_train, X_test, Y_train, Y_test = splitData(x,y)
X_train = X_train 

X_input = Input(batch_shape=(None, X_train.shape[1], X_train.shape[2])) # Input
X_encoder = Bidirectional(LSTM(n_hidden, return_sequences=True), merge_mode='sum')(X_input) # 인코더
y_decoder = Bidirectional(LSTM(n_hidden, return_sequences=True), merge_mode='sum')(X_encoder) # 디코더
y_output = TimeDistributed(Dense(X_train.shape[2]))(y_decoder) # 출력

model = Model(X_input, y_output)

model.compile(loss='mse', optimizer=Adam(lr=0.001))
hist = model.fit(X_train, X_train,
                 epochs=10,
                 batch_size=128)

In [ ]:
model_encoder = Model(model.input, model.layers[1].output)
X_train_encoded = model_encoder.predict(X_train)

In [ ]:
model_encoder.save('./dimensionReduction.h5')
# model.save('iris.h5')